In [2]:
!pip install datasets


In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from nltk.translate.bleu_score import sentence_bleu
import nltk
from torch.optim import AdamW
from transformers import get_scheduler

In [4]:
from datasets import load_dataset

dataset = load_dataset("asas-ai/financial_news", split = "train")

README.md:   0%|          | 0.00/166 [00:00<?, ?B/s]

NMT_Finance_cleaned_pairs.csv:   0%|          | 0.00/8.98M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7560 [00:00<?, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 7560
})

In [6]:
# Extracting sentences

english_sentences = [entry["target"] for entry in dataset]  # List comprehension for English sentences

arabic_sentences = [entry["source"] for entry in dataset]    # List comprehension for Arabic sentences



# Printing the first 5 sentences

print("English Sentences:", english_sentences[:5])  # Print first 5 English sentences

print("Arabic Sentences:", arabic_sentences[:5])

English Sentences: ['The Capital Market Authority Approves Al Sagr Cooperative Insurance Company’s Request to Reduce its Capital', "\u200bThe CMA has issued its resolution approving Al Sagr Cooperative Insurance Company's request to reduce its capital from SAR (400,000,000) to SAR (140,000,000), thus reducing the number of shares from (40,000,000) shares to (14,000,000) shares. This approval is conditional on the Company's extraordinary general assembly approval and completion of the necessary procedures in relation to the applicable regulations. The Company will publish a disclosure document to its shareholders related to the proposed method of capital reduction and the expected effect of such reduction within sufficient time prior to the Extraordinary General Assembly Meeting to enable shareholders to vote on the capital decrease. The CMA's approval of a particular company's application to reduce its capital should never be viewed as an endorsement of the feasibility of the capital d

In [7]:
import torch
from transformers import MarianMTModel, MarianTokenizer

# Load MarianMT model and tokenizer for English to Arabic
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62802, 512, padding_idx=62801)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62802, 512, padding_idx=62801)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [9]:
# Prepare data
# Set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token
input_encodings = tokenizer(english_sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)

target_encodings = tokenizer(arabic_sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)

In [ ]:
# Custom dataset

class TranslationDataset(torch.utils.data.Dataset):

    def __init__(self, input_encodings, target_encodings):

        self.input_encodings = input_encodings

        self.target_encodings = target_encodings



    def __getitem__(self, idx):

        input_ids = self.input_encodings["input_ids"][idx]

        target_ids = self.target_encodings["input_ids"][idx]

        return {"input_ids": input_ids, "labels": target_ids}



    def __len__(self):

        return len(self.input_encodings["input_ids"])

In [11]:
# Prepare DataLoader

dataset = TranslationDataset(input_encodings, target_encodings)

train_loader = DataLoader(dataset, batch_size=8, shuffle=True)



# Prepare optimizer and scheduler

optimizer = AdamW(model.parameters(), lr=1e-4)

num_training_steps = len(train_loader) * 3  # 4 epochs

scheduler = get_scheduler("cosine", optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)

In [12]:
# Main training loop

model.train()

for epoch in range(9):

    total_loss = 0

    for batch in train_loader:

        input_ids = batch["input_ids"].to(device)

        labels = batch["labels"].to(device)



        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, labels=labels)

        loss = outputs.loss

        loss.backward()

        optimizer.step()

        scheduler.step()



        total_loss += loss.item()



    print(f"Epoch [{epoch + 1}/9], Loss: {total_loss / len(train_loader)}")



# Save fine-tuned model

model.save_pretrained("fine_tuned_gpt_model")

Epoch [1/9], Loss: 0.9653874594225454
Epoch [2/9], Loss: 0.28633858111918603
Epoch [3/9], Loss: 0.18646766460505584
Epoch [4/9], Loss: 0.17328946293582992
Epoch [5/9], Loss: 0.17150406172134416
Epoch [6/9], Loss: 0.13488552635289217
Epoch [7/9], Loss: 0.09942086856151935
Epoch [8/9], Loss: 0.09600104241503807
Epoch [9/9], Loss: 0.11218255852403307


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [14]:
# Example translation and BLEU score calculation
def translate_and_evaluate(input_sentence, reference_translation):
    input_text = "translate English to arabic: " + input_sentence
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids)
    translated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    bleu_score = sentence_bleu([nltk.word_tokenize(reference_translation)], nltk.word_tokenize(translated_sentence))
    print(f"Input: {input_sentence}")
    print(f"Translated: {translated_sentence}")
    print(f"Reference: {reference_translation}")
    print(f"BLEU Score: {bleu_score}\n")

In [20]:
# List of input sentences
input_sentences = [
   "The Capital Market Authority approves Tihama Advertising Public Relations and Marketing Company's request to increase its capital by way of Rights Issue",
    "The Capital Market Authority Approves Al Sagr Cooperative Insurance Company’s Request to Reduce its Capital",
     "The Capital Market Authority Announces the Approval on the Registration and Offering shares of Meshkati Trading Company in the Parallel Market",
    "The Capital Market Authority Licenses Middle East Bridge Investment Company to Conduct Managing Investments Activity in the Securities Business and its Completion of the Commencements of Business Requirements"
   
]

# Placeholder reference translations (these should be accurate translations)
reference_translations = [
    "موافقة الهيئة على طلب شركة تهامة للإعلان والعلاقات العامة والتسويق زيادة رأس مالها عن طريق طرح أسهم حقوق أولوية",
    "موافقة الهيئة على طلب شركة الصقر للتأمين التعاوني تخفيض رأس مالها",
    "إعلان من هيئة السوق المالية بشأن الموافقة على تسجيل وطرح أسهم شركة مشكاتي للتجارة في السوق الموازية",
    "الترخيص لشركة جسر الشرق الأوسط للاستثمار في ممارسة نشاط إدارة الاستثمارات في أعمال الأوراق المالية، واستيفائها لمتطلبات بدء ممارسة العمل"

]

for input_sentence, reference_translation in zip(input_sentences, reference_translations):
    translate_and_evaluate(input_sentence, reference_translation)



Input: The Capital Market Authority approves Tihama Advertising Public Relations and Marketing Company's request to increase its capital by way of Rights Issue
Translated: <pad> موافقة هيئة السوق المالية على طلب شركة تهامة للإعلان عن الإعلان العاقات العامة والتسويق زيادة رأس مالها عن طريق طريق طرح أسهم حق حقوق أولوية زيادة رأس رأس مالها عن طريق طرح أسهم طرح حقوق أولوية
Reference: موافقة الهيئة على طلب شركة تهامة للإعلان والعلاقات العامة والتسويق زيادة رأس مالها عن طريق طرح أسهم حقوق أولوية
BLEU Score: 0.3076326722877015

Input: The Capital Market Authority Approves Al Sagr Cooperative Insurance Company’s Request to Reduce its Capital
Translated: <pad> موافقة الهيئة المالية على طلب شركة الصقر للتأمين التعاوني تخفيض رأس مالها تخفيض رأس مالها في تخفيض رأس مالها.
Reference: موافقة الهيئة على طلب شركة الصقر للتأمين التعاوني تخفيض رأس مالها
BLEU Score: 0.37399962732073283

Input: The Capital Market Authority Announces the Approval on the Registration and Offering shares of Meshkati Trading C